In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import pandas as pd

## Test data

In [3]:
test_data = pd.read_pickle('kaggle/test_data.pkl')

In [4]:
import pandas as pd
from tsm.data_utils import time_processing, encode_categories, ordinal2wave
from tsm.data_selector import get_series_past_k_lags

In [6]:
test_data = time_processing(test_data, 'timestamp')

In [7]:
test_data = ordinal2wave('dt_m', test_data)
test_data = ordinal2wave('dt_w', test_data)
test_data = ordinal2wave('dt_d', test_data)
test_data = ordinal2wave('dt_hour', test_data)
test_data = ordinal2wave('dt_day_week', test_data)
test_data = ordinal2wave('dt_day_month', test_data)
test_data = ordinal2wave('dt_week_month', test_data)

Save here for the moment

In [9]:
%whos

Variable                 Type         Data/Info
-----------------------------------------------
encode_categories        function     <function encode_categories at 0x7f94f7ad2488>
get_series_past_k_lags   function     <function get_series_past<...>k_lags at 0x7f954066fd90>
ordinal2wave             function     <function ordinal2wave at 0x7f94f7ad2400>
pd                       module       <module 'pandas' from '/u<...>ages/pandas/__init__.py'>
test_data                DataFrame                row_id  build<...>697600 rows x 37 columns]
time_processing          function     <function time_processing at 0x7f94f7ad2378>


In [10]:
test_data.to_pickle('kaggle/test_data.pkl')

In [11]:
test_data.head()

,row_id,building_id,meter,timestamp,site_id,primary_use,square_feet,year_built,floor_count,air_temperature,...,dt_d_sin,dt_d_cos,dt_hour_sin,dt_hour_cos,dt_day_week_sin,dt_day_week_cos,dt_day_month_sin,dt_day_month_cos,dt_week_month_sin,dt_week_month_cos
0,0,0,0,2017-01-01,0,Education,7432,2008,99,17.799999,...,0.017213,0.999852,0.0,1.0,-2.449294e-16,1.0,0.201299,0.97953,0.951057,0.309017
1,1,1,0,2017-01-01,0,Education,2720,2004,99,17.799999,...,0.017213,0.999852,0.0,1.0,-2.449294e-16,1.0,0.201299,0.97953,0.951057,0.309017
2,2,2,0,2017-01-01,0,Education,5376,1991,99,17.799999,...,0.017213,0.999852,0.0,1.0,-2.449294e-16,1.0,0.201299,0.97953,0.951057,0.309017
3,3,3,0,2017-01-01,0,Education,23685,2002,99,17.799999,...,0.017213,0.999852,0.0,1.0,-2.449294e-16,1.0,0.201299,0.97953,0.951057,0.309017
4,4,4,0,2017-01-01,0,Education,116607,1975,99,17.799999,...,0.017213,0.999852,0.0,1.0,-2.449294e-16,1.0,0.201299,0.97953,0.951057,0.309017


## To encode categories we need to have saved encoder at training time

In [6]:
from sklearn.externals import joblib

/home/Ric/.local/lib/python3.5/site-packages/sklearn/externals/joblib/__init__.py:15: DeprecationWarning: sklearn.externals.joblib is deprecated in 0.21 and will be removed in 0.23. Please import this functionality directly from joblib, which can be installed with: pip install joblib. If this warning is raised when loading pickled models, you may need to re-serialize those models with scikit-learn 0.21+.
  warnings.warn(msg, category=DeprecationWarning)


In [13]:
import numpy as np

In [9]:
encoder = joblib.load('models/LabelEconder_train_data.pkl')

In [14]:
test_data['primary_use'] = encoder.transform(test_data['primary_use']).astype(np.uint8)

In [15]:
test_data.head()

,row_id,building_id,meter,timestamp,site_id,primary_use,square_feet,year_built,floor_count,air_temperature,...,dt_d_sin,dt_d_cos,dt_hour_sin,dt_hour_cos,dt_day_week_sin,dt_day_week_cos,dt_day_month_sin,dt_day_month_cos,dt_week_month_sin,dt_week_month_cos
0,0,0,0,2017-01-01,0,0,7432,2008,99,17.799999,...,0.017213,0.999852,0.0,1.0,-2.449294e-16,1.0,0.201299,0.97953,0.951057,0.309017
1,1,1,0,2017-01-01,0,0,2720,2004,99,17.799999,...,0.017213,0.999852,0.0,1.0,-2.449294e-16,1.0,0.201299,0.97953,0.951057,0.309017
2,2,2,0,2017-01-01,0,0,5376,1991,99,17.799999,...,0.017213,0.999852,0.0,1.0,-2.449294e-16,1.0,0.201299,0.97953,0.951057,0.309017
3,3,3,0,2017-01-01,0,0,23685,2002,99,17.799999,...,0.017213,0.999852,0.0,1.0,-2.449294e-16,1.0,0.201299,0.97953,0.951057,0.309017
4,4,4,0,2017-01-01,0,0,116607,1975,99,17.799999,...,0.017213,0.999852,0.0,1.0,-2.449294e-16,1.0,0.201299,0.97953,0.951057,0.309017


In [16]:
k_air = get_series_past_k_lags(test_data['air_temperature'], k=[1, 2, 3, 6]).fillna(99).reset_index(drop=True)
k_dew = get_series_past_k_lags(test_data['dew_temperature'], k=[1, 2, 3, 6]).fillna(99).reset_index(drop=True)
test_data = pd.concat([test_data.reset_index(drop=True), k_air, k_dew], axis=1, sort=False)

In [19]:
%whos

Variable    Type         Data/Info
----------------------------------
np          module       <module 'numpy' from '/ho<...>kages/numpy/__init__.py'>
pd          module       <module 'pandas' from '/u<...>ages/pandas/__init__.py'>
test_data   DataFrame                row_id  build<...>697600 rows x 45 columns]


In [20]:
k_air_e_001 = test_data.air_temperature.ewm(alpha=0.01).mean().reset_index(drop=True)
k_air_e_001.name = 'k_air_e_001'

In [21]:
test_data = pd.concat([test_data.reset_index(drop=True), k_air_e_001], axis=1, sort=False)

MemoryError: Unable to allocate array with shape (22, 41697600) and data type float64

In [ ]:
del k_air_e_001

In [ ]:
k_air_e_01 = test_data.air_temperature.ewm(alpha=0.1).mean().reset_index(drop=True)
k_air_e_01.name = 'k_air_e_01

In [ ]:
test_data = pd.concat([test_data.reset_index(drop=True), k_air_e_01], axis=1, sort=False)

In [ ]:
del k_air_e_01

In [ ]:
k_air_e_025 = test_data.air_temperature.ewm(alpha=0.25).mean().reset_index(drop=True)
k_air_e_025.name = 'k_air_e_025'

In [ ]:
test_data = pd.concat([test_data.reset_index(drop=True), k_air_e_025], axis=1, sort=False)

In [ ]:
del k_air_e_025

In [ ]:
test_data.to_pickle('kaggle/proc_test_data.pkl')